# Using TensorFlow like NumPy
- TF API revolves around **Tensors**, which flow from operation to operation
- Tensor: multidimensional array like *NP ndarray*
    - import for custom cost functions, custom metrics, custom layers, and more

In [5]:
import tensorflow as tf

tf_arr = tf.constant([[1,2,3,4], [5, 6, 7, 8]])

In [6]:
tf_arr.shape

TensorShape([2, 4])

In [8]:
tf_arr.dtype

tf.int32

In [10]:
tf_arr[:, :1]

<tf.Tensor: shape=(2, 1), dtype=int32, numpy=
array([[1],
       [5]], dtype=int32)>

In [12]:
tf_arr + 1000

<tf.Tensor: shape=(2, 4), dtype=int32, numpy=
array([[1001, 1002, 1003, 1004],
       [1005, 1006, 1007, 1008]], dtype=int32)>

In [15]:
tf.square(tf_arr)

<tf.Tensor: shape=(2, 4), dtype=int32, numpy=
array([[ 1,  4,  9, 16],
       [25, 36, 49, 64]], dtype=int32)>

In [23]:
tf_arr @ tf.transpose(tf_arr) # '@' means matrix multiplication in Python 3.5 and higher

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 30,  70],
       [ 70, 174]], dtype=int32)>

## <font color="orange">Keras' Low-Level API</font>

In [24]:
from tensorflow import keras

K = keras.backend
K.square(tf_arr) + 10

<tf.Tensor: shape=(2, 4), dtype=int32, numpy=
array([[11, 14, 19, 26],
       [35, 46, 59, 74]], dtype=int32)>

## <font color='orange'>Tensors and NumPy ops</font>
- sidenote: NumPy uses 64-bit precision, while TF uses 32-bit precision. 32-bits is enough for NNs and makes it run faster w/ less memory, so make sure to set **dtype=float32** when converting from TF -> NP array

In [30]:
import numpy as np

a = np.array([1, 3, 5])
tf.constant(a)

<tf.Tensor: shape=(3,), dtype=int64, numpy=array([1, 3, 5])>

In [27]:
tf.square(a)

<tf.Tensor: shape=(3,), dtype=int64, numpy=array([ 1,  9, 25])>

In [28]:
np.square(a)

array([ 1,  9, 25])

In [36]:
tf.constant(2.) + tf.constant(40, dtype=tf.float32) # TF does NOT do type conversions automatically

<tf.Tensor: shape=(), dtype=float32, numpy=42.0>

### <font color="red">Variables</font>
- TF variables are immutable. The TF operations so far create new tensors, not modify them
    - Need to use mutables arrays for weights in a NN because of backprop
- use **assign()** method to change values in tensors

In [39]:
v = tf.Variable([[1, 2, 3], [4, 5, 6]])
v[0, 1].assign(1000)
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=int32, numpy=
array([[   1, 1000,    3],
       [   4,    5,    6]], dtype=int32)>

### TF Data Structures
- tensors
- sparse tensors
    - efficient representation of tensors containing mostly zeros
- tensor arrays
    - list of tensors
- ragged tensors
    - static lists of lists of tensors
- string tensors
    - regular tensors of type tf.string
- sets
    - representated as regular or sparse tensors
    - tf.constant([1, 2], [3, 4]) is are sets {1, 2} and {3, 4}
- queues

## <font color="orange">Customizing Models and Training Algorithms</font>
### <font color="red">Custom Loss Functions</font>
- use one if its not implemented in TF already and if it could benefit your model
- Example scenario: noisy training set, MSE penalizes outliers too much and cause the model to imprecise, MAE doesn't penalize outliers as much but takes very long to converge and the model might still be imprecise. Now's a good time to use Huber loss

In [40]:
def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2
    linear_loss = tf.abs(error) - 0.5
    return tf.where(is_small_error, square_loss, linear_loss) #return the whole array of losses per instance rather than the mean loss

the model will use this function to compute the loss and perform a GD step and track the total loss since the start of the epoch, and then display the mean loss

### <font color="red">Saving and Loading Models That Contain Custom Components</font>
- Keras saves the name of the function
    - Will need to provide a dictionary that maps the name to the function
- Keras saves the name of custom objects so a dictionary that maps the names to their functions will be needed

In [41]:
def create_huber(threshold=1.0): #function that allows for threshold customization because the loss function above's threshold is only -1 and 1
    def huber_fn(y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < threshold
        squared_loss = tf.square(error) / 2
        linear_loss = tf.abs(error) - 0.5
        return tf.where(is_small_error, square_loss, linear_loss)
    return huber_f

create_huber(2.0) #threshold is not save by the model so it must be specified

subclassing Keras.losses.Loss class to allow for saving the threshold:

In [42]:
class HuberLoss(keras.losses.Loss):
    def __init__(self, threshold=1.0, **kwargs):
        self.threshold = threshold
        super().__init__(**kwargs)
    def call(self, y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < self.threshold
        squared_loss = tf.square(error) / 2
        linear_loss = self.threshold * tf.abs(error) - self.threshold**2 / 2
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}

- \*\*kwargs is passed to the parent constructor, which handles stangard hyperparameters (name of the loss, reduction algorithm used to aggregate the individual instance losses)
- call() method takes the predictions and labels to compute the loss
- get_config() method return a dictionary mapping each hyperparameter naem to its value. returns parent's config and its own config

## <font color="orange">Custom Activation Functions, Initializers, Regularizers, and Constraints</font>

In [43]:
def my_softplus(z):
    return tf.math.log(tf.exp(z) + 1.0)

def my_glorot_initializer(shape, dtype=tf.float32):
    stddev = tf.sqrt(2. / (shape[0] + shape[1]))
    return tf.random.normal(shape, stddev=stddev, dtype=dtype)

def my_l1_regularlizer(weights):
    return tf.reduce_sum(tf.abs(0.01 * weights))

def my_positive_weights(weights):
    return tf.where(weights < 0., tf.zeros_like(weights), weights)

In [46]:
from tensorflow import keras

layer = keras.layers.Dense(30, activation=my_softplus, kernel_initializer=my_glorot_initializer, kernel_regularizer=my_l1_regularlizer,
                         kernel_constraint=my_positive_weights)